In [1]:
import gseapy as gp

In [2]:
gp.get_library_name()

['ARCHS4_Cell-lines',
 'ARCHS4_IDG_Coexp',
 'ARCHS4_Kinases_Coexp',
 'ARCHS4_TFs_Coexp',
 'ARCHS4_Tissues',
 'Achilles_fitness_decrease',
 'Achilles_fitness_increase',
 'Aging_Perturbations_from_GEO_down',
 'Aging_Perturbations_from_GEO_up',
 'Allen_Brain_Atlas_down',
 'Allen_Brain_Atlas_up',
 'BioCarta_2013',
 'BioCarta_2015',
 'BioCarta_2016',
 'BioPlanet_2019',
 'BioPlex_2017',
 'CORUM',
 'Cancer_Cell_Line_Encyclopedia',
 'ChEA_2013',
 'ChEA_2015',
 'ChEA_2016',
 'Chromosome_Location',
 'Chromosome_Location_hg19',
 'ClinVar_2019',
 'DSigDB',
 'Data_Acquisition_Method_Most_Popular_Genes',
 'DepMap_WG_CRISPR_Screens_Broad_CellLines_2019',
 'DepMap_WG_CRISPR_Screens_Sanger_CellLines_2019',
 'DisGeNET',
 'Disease_Perturbations_from_GEO_down',
 'Disease_Perturbations_from_GEO_up',
 'Disease_Signatures_from_GEO_down_2014',
 'Disease_Signatures_from_GEO_up_2014',
 'DrugMatrix',
 'Drug_Perturbations_from_GEO_2014',
 'Drug_Perturbations_from_GEO_down',
 'Drug_Perturbations_from_GEO_up',
 'EN

In [5]:
genelist = ['TIMP1', 'CASP3', 'PLA2G5', 'CDC6', 'TIMP4', 'INHBB', 'HOMER1',
       'PDLIM3', 'RPRM', 'DKK1', 'DYNLT3', 'NET1', 'NET1', 'CCNA1', 'PDGFRA',
       'NSUN5', 'NSUN5', 'PYGB', 'SLC43A3', 'TAGLN2', 'CDK2', 'ID4', 'ATRX',
       'SH2B2', 'SFRP4', 'FAM3C', 'ACOX2', 'THBS2', 'ZNF609', 'SFRP1',
       'IGFBP3', 'MGMT', 'FZD3', 'CCNE1', 'LGALS8', 'LGALS8', 'TP53', 'THBS3',
       'IDH1', 'MDK', 'DCTD', 'PRKAR2B', 'KLF13', 'POLE3', 'IGFBP2', 'IGFBP2',
       'CCND1', 'CDKN2A', 'PDPN', 'ADM', 'FZD7', 'GHR', 'EZH2', 'TFRC',
       'RGS14', 'PIK3CA', 'VEGFA', 'REPS2', 'MT1M', 'CDKN1C', 'STMN1', 'IRS2',
       'HSPA1B', 'LIFR', 'G6PC3', 'GADD45G', 'FBXO17', 'PTPRN', 'WAC',
       'STAG3L4', 'EFEMP2', 'EFEMP2', 'PPARGC1A', 'EGFR', 'MT1E']

In [10]:
enr = gp.enrichr(gene_list=genelist,
                 gene_sets=['GO_Biological_Process_2018',
                            'GO_Cellular_Component_2018',
                            'GO_Molecular_Function_2018'],
#                  outdir='test/enrichr_kegg',
                 no_plot=True,
                 cutoff=0.001 # test dataset, use lower value from range(0,1)
                )

In [101]:
tumor_dfs["PJ016"].obs["PHATE 1"]

0       0.035781
1       0.044900
2      -0.012389
3       0.037923
4      -0.010752
          ...   
3080   -0.013115
3081   -0.007255
3082    0.000888
3083   -0.013590
3084   -0.000256
Name: PHATE 1, Length: 3085, dtype: float64

In [11]:
enr.results

,Adjusted P-value,Combined Score,Gene_set,Genes,Odds Ratio,Old Adjusted P-value,Old P-value,Overlap,P-value,Term
0,0.000021,119.406906,GO_Biological_Process_2018,PDGFRA;CDKN2A;IGFBP3;LIFR;IRS2;CDC6;EGFR;VEGFA...,6.177606,0,0,16/740,4.032042e-09,regulation of cell proliferation (GO:0042127)
1,0.000029,139.994319,GO_Biological_Process_2018,PDGFRA;FZD3;MGMT;DKK1;EGFR;VEGFA;SFRP1;PIK3CA;...,7.658321,0,0,13/485,1.151025e-08,negative regulation of apoptotic process (GO:0...
2,0.000044,125.098116,GO_Biological_Process_2018,PDGFRA;LIFR;IRS2;CDC6;DKK1;EGFR;VEGFA;SFRP1;AC...,7.156620,0,0,13/519,2.561581e-08,positive regulation of cellular process (GO:00...
3,0.000036,1985.799593,GO_Biological_Process_2018,CCND1;CDK2;WAC;TP53,114.285714,0,0,4/10,2.843212e-08,G1 DNA damage checkpoint (GO:0044783)
4,0.000084,310.781696,GO_Biological_Process_2018,CCNA1;CCND1;CDKN2A;CCNE1;POLE3;CDK2;CDC6,19.047619,0,0,7/105,8.204163e-08,G1/S transition of mitotic cell cycle (GO:0000...
...,...,...,...,...,...,...,...,...,...,...
1375,1.000000,0.459770,GO_Molecular_Function_2018,TP53,1.006036,0,0,1/284,6.331734e-01,"transcriptional activator activity, RNA polyme..."
1376,1.000000,0.431396,GO_Molecular_Function_2018,TP53,0.978474,0,0,1/292,6.434657e-01,transcription regulatory region sequence-speci...
1377,1.000000,0.206478,GO_Molecular_Function_2018,PPARGC1A,0.725163,0,0,1/394,7.522146e-01,sequence-specific DNA binding (GO:0043565)
1378,1.000000,0.177485,GO_Molecular_Function_2018,FBXO17,0.685166,0,0,1/417,7.717931e-01,ubiquitin-protein transferase activity (GO:000...
